In [1]:
import requests
import base64
import pandas as pd

# Adjust display options
pd.set_option('display.max_rows', None)   # Show all rows
pd.set_option('display.max_columns', None)   # Show all columns

ModuleNotFoundError: No module named 'requests'

In [ ]:
CLIENT_ID = 'fe625cd1e17f45cca39c0fbb4b9b4313'
CLIENT_SECRET = '5d59a2b5904441c39e26384d1399be1c'

# Base64 encode the client ID and client secret
client_credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"
client_credentials_base64 = base64.b64encode(client_credentials.encode())

# Request the access token
token_url = 'https://accounts.spotify.com/api/token'
headers = {
    'Authorization': f'Basic {client_credentials_base64.decode()}'
}
data = {
    'grant_type': 'client_credentials'
}
response = requests.post(token_url, data=data, headers=headers)

if response.status_code == 200:
    access_token = response.json()['access_token']
    print("Access token obtained successfully.")
else:
    print("Error obtaining access token.")
    exit()

Access token obtained successfully.


In [ ]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth

def get_trending_playlist_data(playlist_id, access_token):
    # Set up Spotipy with the access token
    sp = spotipy.Spotify(auth=access_token)

    # Get the tracks from the playlist
    playlist_tracks = sp.playlist_tracks(playlist_id, fields='items(track(id, name, artists, album(id, name)))', limit = 90)

    # Extract relevant information and store in a list of dictionaries
    music_data = []
    for track_info in playlist_tracks['items']:
        track = track_info['track']
        track_name = track['name']
        artists = ', '.join([artist['name'] for artist in track['artists']])
        album_name = track['album']['name']
        album_id = track['album']['id']
        track_id = track['id']

        # Get audio features for the track
        audio_features = sp.audio_features(track_id)[0] if track_id != 'Not available' else None

        # Get release date of the album
        try:
            album_info = sp.album(album_id) if album_id != 'Not available' else None
            release_date = album_info['release_date'] if album_info else None
        except:
            release_date = None

        # Get popularity of the track
        try:
            track_info = sp.track(track_id) if track_id != 'Not available' else None
            popularity = track_info['popularity'] if track_info else None
        except:
            popularity = None

        # Add additional track information to the track data
        track_data = {
            'Track Name': track_name,
            'Artists': artists,
            'Album Name': album_name,
            'Album ID': album_id,
            'Track ID': track_id,
            'Popularity': popularity,
            'Release Date': release_date,
            'Duration (ms)': audio_features['duration_ms'] if audio_features else None,
            'Explicit': track_info.get('explicit', None),
            'External URLs': track_info.get('external_urls', {}).get('spotify', None),
            'Danceability': audio_features['danceability'] if audio_features else None,
            'Energy': audio_features['energy'] if audio_features else None,
            'Key': audio_features['key'] if audio_features else None,
            'Loudness': audio_features['loudness'] if audio_features else None,
            'Mode': audio_features['mode'] if audio_features else None,
            'Speechiness': audio_features['speechiness'] if audio_features else None,
            'Acousticness': audio_features['acousticness'] if audio_features else None,
            'Instrumentalness': audio_features['instrumentalness'] if audio_features else None,
            'Liveness': audio_features['liveness'] if audio_features else None,
            'Valence': audio_features['valence'] if audio_features else None,
            'Tempo': audio_features['tempo'] if audio_features else None,
            # Add more attributes as needed
        }

        music_data.append(track_data)

    # Create a pandas DataFrame from the list of dictionaries
    df = pd.DataFrame(music_data)

    return df

In [ ]:
#playlist_id = '3rNqxghsSqsloXkPly9FIU'
playlist_id = '3xpqwk1Kqjsc5q9b2HL7WN'
#playlist_id = '37i9dQZF1DXcBWIGoYBM5M' #Today's Top Hits
#playlist_id = '37i9dQZEVXbMDoHDwVN2tF' #Global Top 50

# Call the function to get the music data from the playlist and store it in a DataFrame
music_df = get_trending_playlist_data(playlist_id, access_token)
music_df.head()

,Track Name,Artists,Album Name,Album ID,Track ID,Popularity,Release Date,Duration (ms),Explicit,External URLs,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
0,The Boys,Girls' Generation,The Boys,5R0SllxstxihA5EYBPualh,7eNZS973JNO67aP8mVGU6i,41,2011-01-01,226890,False,https://open.spotify.com/track/7eNZS973JNO67aP...,0.885,0.941,1,-0.587,0,0.074,0.19,0,0.218,0.923,114.987


In [ ]:
try:
    playlist_id = '3xpqwk1Kqjsc5q9b2HL7WN'
    get_trending_playlist_data(playlist_id, access_token)
    results = sp.search(q='weezer', limit=1)
except spotipy.SpotifyException as e:
    if e.http_status == 429:
        wait_time = int(e.headers['Retry-After'])
        print(f"Rate limit exceeded. Waiting for {wait_time} seconds.")
        time.sleep(wait_time)

Max Retries reached


KeyError: 'Retry-After'

In [ ]:
#check for null and duplicates
print(music_df.isna().sum())
print( "No. of duplicates:" + str(music_df.duplicated().sum()))

Track Name          0
Artists             0
Album Name          0
Album ID            0
Track ID            0
Popularity          0
Release Date        0
Duration (ms)       0
Explicit            0
External URLs       0
Danceability        0
Energy              0
Key                 0
Loudness            0
Mode                0
Speechiness         0
Acousticness        0
Instrumentalness    0
Liveness            0
Valence             0
Tempo               0
dtype: int64
No. of duplicates:0


In [ ]:
#Building the ML Model
import pandas as pd
import numpy as np
def dummy_npwarn_decorator_factory():
  def npwarn_decorator(x):
    return x
  return npwarn_decorator
np._no_nep50_warning = getattr(np, '_no_nep50_warning', dummy_npwarn_decorator_factory)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity

data = music_df

In [ ]:
# Function to calculate weighted popularity scores based on release date
def calculate_weighted_popularity(release_date):
    # Convert the release date to datetime object
    release_date = datetime.strptime(release_date, '%Y-%m-%d')

    # Calculate the time span between release date and today's date
    time_span = datetime.now() - release_date

    # Calculate the weighted popularity score based on time span (e.g., more recent releases have higher weight)
    weight = 1 / (time_span.days + 1)
    return weight

In [ ]:
# Normalize the music features using Min-Max scaling
scaler = MinMaxScaler()
music_features = music_df[['Danceability', 'Energy', 'Key', 
                           'Loudness', 'Mode', 'Speechiness', 'Acousticness',
                           'Instrumentalness', 'Liveness', 'Valence', 'Tempo']].values
music_features_scaled = scaler.fit_transform(music_features)

In [ ]:
# a function to get content-based recommendations based on music features
def content_based_recommendations(input_song_name, num_recommendations=5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Get the index of the input song in the music DataFrame
    input_song_index = music_df[music_df['Track Name'] == input_song_name].index[0]

    # Calculate the similarity scores based on music features (cosine similarity)
    similarity_scores = cosine_similarity([music_features_scaled[input_song_index]], music_features_scaled)

    # Get the indices of the most similar songs by descending order of similarity scores
    similar_song_indices = similarity_scores.argsort()[0][::-1][1:num_recommendations + 1]

    # Get the names of the most similar songs based on content-based filtering
    content_based_recommendations = music_df.iloc[similar_song_indices][['Track Name', 'Artists', 'Album Name', 'Release Date', 'Popularity']]

    return content_based_recommendations

In [ ]:
# a function to get hybrid recommendations based on weighted popularity
def hybrid_rec(input_song_name, num_recommendations=5, alpha=0.5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Get content-based recommendations
    content_based_rec = content_based_recommendations(input_song_name, num_recommendations)

    # Get the popularity score of the input song
    popularity_score = music_df.loc[music_df['Track Name'] == input_song_name, 'Popularity'].values[0]

    # Calculate the weighted popularity score
    weighted_popularity_score = popularity_score * calculate_weighted_popularity(music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0])

    # Combine content-based and popularity-based recommendations based on weighted popularity
    hybrid_recommendations = content_based_rec
    #hybrid_recommendations = hybrid_recommendations.append({
       # 'Track Name': input_song_name,
       # 'Artists': music_df.loc[music_df['Track Name'] == input_song_name, 'Artists'].values[0],
       # 'Album Name': music_df.loc[music_df['Track Name'] == input_song_name, 'Album Name'].values[0],
      #  'Release Date': music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0],
       # 'Popularity': weighted_popularity_score
   # }, ignore_index=True)
    new_row = pd.DataFrame([{
    'Track Name': input_song_name,
    'Artists': music_df.loc[music_df['Track Name'] == input_song_name, 'Artists'].values[0],
    'Album Name': music_df.loc[music_df['Track Name'] == input_song_name, 'Album Name'].values[0],
    'Release Date': music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0],
    'Popularity': weighted_popularity_score
     }])

    hybrid_recommendations = pd.concat([hybrid_recommendations, new_row], ignore_index=True)


    # Sort the hybrid recommendations based on weighted popularity score
    hybrid_recommendations = hybrid_recommendations.sort_values(by='Popularity', ascending=False)

    # Remove the input song from the recommendations
    hybrid_recommendations = hybrid_recommendations[hybrid_recommendations['Track Name'] != input_song_name]


    return hybrid_recommendations

In [ ]:
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")


In [ ]:
song_name = '小胡同'
#song_name = 'MELTDOWN (feat. Drake)'
recommendations = hybrid_rec(song_name, num_recommendations=5)
print(f"Hybrid recommended songs for '{song_name}':")
recommendations

Hybrid recommended songs for '小胡同':


,Track Name,Artists,Album Name,Release Date,Popularity
4,你的轮廓,叶琼琳,你的轮廓,2021-01-22,49.0
2,hard to let you go,Sherman Zhuo,hard to let you go,2023-03-31,10.0
0,就让这大雨全都落下,Joey Yung,就让这大雨全都落下,2022-07-22,1.0
1,予你,隊長,予你,2022-03-19,0.0
3,单人券,张齐山DanieL,单人券,2022-05-06,0.0
